In [ ]:
# collaborative filtering


In [1]:
from collections import OrderedDict

filePath = "/home/chowder/Documents/AiLearning/codingChallenges/kaggle/" + \
    "competitions/MillionSongDataset/MillionSongSubset/" + \
    "train_triplets.txt"



tripletsFile = open(filePath, "r")

rowDicts = []
triplets = OrderedDict()
dictToAdd = {}
currUserName = ""
lineCount = 0
for line in tripletsFile.readlines():
    lineCount += 1
    splitLine = line.split()
    # rowDicts.append({"user": splitLine[0], "track": splitLine[1], "numListens": splitLine[2]})

    if lineCount == 1:
        currUserName = splitLine[0]

    
    if currUserName != splitLine[0]:
        triplets[currUserName] = dictToAdd
        dictToAdd = {}
        currUserName = splitLine[0]
    
    dictToAdd[splitLine[1]] = int(splitLine[2])

In [3]:
# list of distinct users
userList = [user for user in triplets]
print(len(userList))




# list of distinct songs, sorted for easier search 
songList = sorted(list(set([song for user in triplets for song in triplets[user]])))
print(len(songList))

1019317
384546


In [4]:
from bisect import bisect_left

# make the data, row, and column lists
row = []
col = []
data = []
for userI in range(len(userList)):
    songDictForUser = triplets[userList[userI]]
    for song in songDictForUser:
        row.append(userI)
        col.append(bisect_left(songList,song))
        data.append(songDictForUser[song])
        if len(data) %3_000_000 == 0 :
            print(len(data))



3000000
6000000
9000000
12000000
15000000
18000000
21000000
24000000
27000000
30000000
33000000
36000000
39000000
42000000
45000000
48000000


In [27]:
# using multiprocessing
print(len(data)) # in 14 minutes and 45 seconds
print(len(row)) 
print(len(col)) 

48373547
48373547
48373547


In [33]:
from scipy.sparse import coo_matrix

numInteractions = 100_000

# make a sparse coo scipy matrix
cooMatrix = coo_matrix((data[0:numInteractions], (row[0:numInteractions], col[0:numInteractions])), 
                        shape=(len(set(userList[0:numInteractions])),len(songList)))


In [34]:
print(cooMatrix)

  (0, 6347)	1
  (0, 9365)	1
  (0, 16962)	2
  (0, 19513)	1
  (0, 19536)	1
  (0, 24689)	1
  (0, 27760)	2
  (0, 29262)	1
  (0, 30363)	1
  (0, 30512)	1
  (0, 31127)	1
  (0, 40407)	1
  (0, 48122)	1
  (0, 50216)	5
  (0, 62690)	1
  (0, 64062)	1
  (0, 68251)	1
  (0, 70743)	1
  (0, 72816)	1
  (0, 77813)	1
  (0, 83403)	1
  (0, 84283)	1
  (0, 90798)	1
  (0, 91886)	1
  (0, 95332)	1
  :	:
  (2040, 359206)	1
  (2040, 366894)	5
  (2040, 367161)	1
  (2040, 374782)	1
  (2040, 376642)	1
  (2040, 378736)	2
  (2040, 379818)	5
  (2040, 380605)	4
  (2041, 1916)	2
  (2041, 23720)	3
  (2041, 25597)	8
  (2041, 38857)	3
  (2041, 49024)	3
  (2041, 65692)	16
  (2041, 67917)	18
  (2041, 78067)	4
  (2041, 86227)	2
  (2041, 86316)	4
  (2041, 122075)	5
  (2041, 137435)	3
  (2041, 158094)	3
  (2041, 211477)	4
  (2041, 241533)	8
  (2041, 259090)	3
  (2041, 279876)	18


In [35]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp')
model.fit(cooMatrix, epochs=10, num_threads=12)

train_precision = precision_at_k(model, cooMatrix, k=10).mean()
test_precision = precision_at_k(model, cooMatrix, k=10).mean()

train_auc = auc_score(model, cooMatrix).mean()
test_auc = auc_score(model, cooMatrix).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
# bpr
# 16.5 sec 20 epochs, 10k rows 172 users
# 19.1 sec 10 epochs, 10k rows 172 users
# 22.5 sec 1 epoch, 10k rows 172 users
# 3m 32s 10 epochs, 100k rows, 2041 users
# 3m 48s 1 epoch, 100k rows, 2041 users
# total is 48M rows, 1M users est. 26 hours for 10 epochs
# warp
# 16.5 sec 1 epoch, 10k rows 172 users
# 16.1 sec 10 epochs, 10k rows 172 users
# 16.2 sec 20 epochs, 10k rows 172 users
# 3m 10s 1 epoch, 100k rows, 2041 users
# 2m 51s 10 epochs, 100k rows, 2041 users
# 2m 50s 10 epochs, 100k rows, 2041 users 10 threads


Precision: train 0.14, test 0.14.
AUC: train 0.99, test 0.99.


In [36]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp')
model.fit(cooMatrix, epochs=10, num_threads=12)

In [39]:
%time train_precision = precision_at_k(model, cooMatrix, k=10, num_threads=12).mean()
%time test_precision = precision_at_k(model, cooMatrix, k=10, num_threads=12).mean()
# 1m 24s 100k rows, 2041 users
# 1m 25s 100k rows, 2041 users 12 threads



CPU times: user 44.4 s, sys: 172 ms, total: 44.6 s
Wall time: 42.7 s
CPU times: user 44.5 s, sys: 112 ms, total: 44.6 s
Wall time: 42.7 s


In [40]:
%time train_auc = auc_score(model, cooMatrix, num_threads=12).mean()
test_auc = auc_score(model, cooMatrix, num_threads=12).mean()
# 1m 24s 100k rows, 2041 users 42 sec each
# 1m 24s 100k rows, 2041 users 42 sec each 12 threads


CPU times: user 43.9 s, sys: 95.9 ms, total: 44 s
Wall time: 42.2 s


In [41]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.14, test 0.14.
AUC: train 0.99, test 0.99.


In [5]:
print(len(set(userList)))

1019317


In [7]:
# build dataset
from lightfm.data import Dataset

lfmDataset = Dataset()
lfmDataset.fit((x for x in userList),(x for x in songList))

dataset.fit((x['User-ID'] for x in get_ratings()),
            (x['ISBN'] for x in get_ratings()))

In [9]:
num_users, num_items = lfmDataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 1019317, num_items 384546.


In [ ]:
# if we want to add more we can call fit partial
# dataset.fit_partial(items=(x['ISBN'] for x in get_book_features()),
#                     item_features=(x['Book-Author'] for x in get_book_features()))